In [ ]:
%%capture
import warnings
warnings.filterwarnings("ignore")

import calitp_data_analysis.magics

import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
import _new_ct_report_utils as utils
from update_vars import GTFS_DATA_DICT, analysis_month, file_name 

# Maps
import branca
import branca.colormap as cm
from shared_utils import webmap_utils
import geopandas as gpd

# Display
from great_tables import GT
from IPython.display import HTML, Image, Markdown, display, display_html
from slugify import slugify

In [ ]:
import google.auth
credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [ ]:
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas

@cache
def gcs_pandas():
    return GCSPandas()

In [ ]:
district = '07-Los Angeles / Ventura'

In [ ]:
district_number = int(district[:2])

In [ ]:
# %%capture_parameters
# district

In [ ]:
GCS_PATH = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/"

In [ ]:
# Files for webmaps
boundary_file = f"district_{district_number}_boundary"
transit_routes_file = f"district_{district_number}_transit_routes"
shn_file = f"district_{district_number}_shn"
transit_shn_file = f"district_{district_number}_transit_routes_shn"

In [ ]:
transit_route_shs_gdf = (
    utils.load_shn_transit_routes(district = district,
                                 pct = 15)
)

In [ ]:
fct_monthly_routes_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}.parquet"

In [ ]:
operator_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet"

In [ ]:
operator_df = gcs_pandas().read_parquet(
    operator_summary_url,
    filters=[
        ("Caltrans District", "==", district),
        ("Date", "==", pd.Timestamp(analysis_month)),
        ("Day Type", "==", "Weekday")
    ],
)

In [ ]:
fct_monthly_route_df = gpd.read_parquet(fct_monthly_routes_url,
                                        filters=[[("Caltrans District", "==", district)]],
                                        storage_options={"token": credentials.token})

In [ ]:
fct_monthly_route_df = utils.prep_gdf(fct_monthly_route_df)

In [ ]:
district_gdf = utils.load_ct_district(district_number)

In [ ]:
shn_gdf =  utils.load_buffered_shn_map(district_number)

# {district}
These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

## District Overview

In [ ]:
try:
    operator_df2 = utils.create_summary_table(operator_df)
except:
    pass

In [ ]:
try:
    display(
    GT(operator_df2
        )
        .tab_header(title=f"District {district} GTFS summary stats")
    )
except:
    pass

## Routes within the District

In [ ]:
color_map = cm.linear.Spectral_11.scale()

In [ ]:
try:
    color_map = branca.colormap.LinearColormap(
    colors=color_map.colors[7:], vmin=0, vmax=fct_monthly_route_df.Number.max()
)
except:
    pass

In [ ]:
try:
    district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
    overwrite = True
)
except:
    pass

In [ ]:
try:
    transit_routes = webmap_utils.set_state_export(
    fct_monthly_route_df,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes",
    cmap=color_map,
    color_col="Number",
    existing_state=district_map,
    overwrite = True
    
)
except:
    pass

In [ ]:
try:
    webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")
except:
    pass

In [ ]:
try:
    webmap_utils.display_spa_map(transit_routes["spa_link"])
except:
    pass

## Transit Routes on the State Highway Network
**Only transit routes that have 15% or more if its length on one or more State Highway Network routes are included**

In [ ]:
color_map2 = cm.linear.RdYlBu_11.scale()

In [ ]:
color_map2 = branca.colormap.LinearColormap(
    colors=color_map2.colors[7:], vmin=0, vmax=100
)

In [ ]:
try:
    shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder = "caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type='state_highway_network',
    overwrite = True
)
except:
    pass

In [ ]:
try:
    transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Percentage of Transit Route on SHN Across All Districts",
    existing_state=shn_map,
    legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg",
    overwrite = True
    
)
except:
    pass

In [ ]:
try:
    webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Open Routes on State Highway System Map")
except:
    pass

In [ ]:
try:
    webmap_utils.display_spa_map(transit_shn_map["spa_link"])
except:
    pass

In [ ]:
try:
    display(GT(
    transit_route_shs_gdf.drop(columns = ["geometry"]).sort_values(
        by=[
            "Analysis Name",
            "Percentage of Transit Route on SHN Across All Districts",
        ],
        ascending=[True, False],
    )
))
except:
    pass

## GTFS Stats by Operator

In [40]:
try:
    gtfs_table = utils.create_operator_table(operator_df)
except:
    pass

In [41]:
try:
    display((
    GT(gtfs_table.sort_values("Daily Trips", ascending=False))
    .fmt_integer(
        columns=[
            c
            for c in gtfs_table.columns
            if c not in ["Operator"]
        ]
    )
    .tab_header(
        title=f"District {district}",
        subtitle="Daily GTFS schedule statistics for Weekday by operator",
    )
    .cols_align(
        columns=[
            c
            for c in gtfs_table.columns
        ],
        align="center",
    )
))
except:
    pass

GT(_tbl_data=                                             Operator  Daily Trips  \
8   Los Angeles County Metropolitan Transportation...      13179.8   
10                                City of Los Angeles       3064.1   
6                                  Long Beach Transit       1867.0   
7                                    Foothill Transit       1854.3   
2                                City of Santa Monica       1300.7   
20  Los Angeles County Metropolitan Transportation...       1238.6   
9   Ventura County (VCTC, Gold Coast, Cities of Ca...       1152.1   
1                   Antelope Valley Transit Authority        529.9   
23                                   City of Torrance        453.0   
0                                    City of Torrance        434.0   
4                               City of Santa Clarita        425.9   
13                                   City of Pasadena        401.2   
11                                 City of Montebello        391.1   
22                                    City of Norwalk        315.0   
12                                    City of Gardena        282.0   
5                                    City of Commerce        113.7   
3                              City of West Hollywood         99.3   
14                                    City of Burbank         99.0   
18                              City of Redondo Beach         98.1   
17                                    City of Arcadia         89.0   
16                              City of Monterey Park         62.0   
21                                     City of Downey         55.0   
19                                   City of Cerritos         42.0   
15                                   City of Lawndale         30.0   

      # Routes    # Shapes       # Stops  
8   107.826087  613.260870  11838.521739  
10   65.347826  111.565217   2748.391304  
6    35.826087  101.391304   1868.260870  
7    30.869565   78.260870   1717.652174  
2    14.304348   44.304348    779.173913  
20    6.000000   12.000000    111.000000  
9    51.318182  106.545455   1225.772727  
1    19.130435   41.130435    826.434783  
23   12.000000   43.000000    784.000000  
0    11.666667   41.333333    777.000000  
4    38.000000  103.909091    728.409091  
13   10.565217   31.130435    437.217391  
11    7.869565   38.608696    815.130435  
22    6.000000   26.000000    442.000000  
12    5.217391   24.391304    418.000000  
5     7.000000    7.000000    225.000000  
3     3.571429    5.142857     88.428571  
14    2.000000    2.000000     47.000000  
18    2.000000    5.000000    201.000000  
17    3.000000    5.000000     76.000000  
16    4.000000    4.000000     88.000000  
21    6.000000   13.000000     95.000000  
19    2.000000    6.000000     21.000000  
15    2.000000    2.000000     43.000000  , _body=<great_tables._gt_data.Body object at 0x7d10f18024d0>, _boxhead=Boxhead([ColInfo(var='Operator', type=<ColInfoTypeEnum.default: 1>, column_label='Operator', column_align='center', column_width=None), ColInfo(var='Daily Trips', type=<ColInfoTypeEnum.default: 1>, column_label='Daily Trips', column_align='center', column_width=None), ColInfo(var='# Routes', type=<ColInfoTypeEnum.default: 1>, column_label='# Routes', column_align='center', column_width=None), ColInfo(var='# Shapes', type=<ColInfoTypeEnum.default: 1>, column_label='# Shapes', column_align='center', column_width=None), ColInfo(var='# Stops', type=<ColInfoTypeEnum.default: 1>, column_label='# Stops', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7d10f1080a90>, _spanners=Spanners([]), _heading=Heading(title='District 07-Los Angeles / Ventura', subtitle='Daily GTFS schedule statistics for Weekday by operator', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7d10f10824d0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7d10f1082850>], _substitution